In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import torch
import copy
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import Tensor

In [2]:
train = pd.read_csv('dataAll.csv')
test = pd.read_csv('testAll.csv')
train_data=train.iloc[:,:30].applymap(lambda x: x+126.23)
train_loc = train.iloc[:,30:]

x_test = test.applymap(lambda x: x+126.23)
x_test = torch.from_numpy(
       x_test.values.astype(np.float32))

In [3]:
train_feat = torch.from_numpy(
        train_data.values.astype(np.float32))
train_labels = torch.from_numpy(
        train_loc.values.astype(np.float32))

In [4]:
num_train = int(0.8 * train_feat.shape[0]) # 划分训练样本和验证集样本
indices = np.arange( train_feat.shape[0])
np.random.shuffle(indices)  # shuffle 顺序
train_indices = indices[:num_train]
valid_indices = indices[num_train:]

train_features = train_feat[train_indices,:]
train_label = train_labels[train_indices,:]
valid_features  = train_feat[valid_indices,:]
valid_labels = train_labels[valid_indices,:]


In [5]:
feat_dim = train_data.shape[0]

In [6]:
def error(y_test,y_pre):
    y_test = np.array(y_test)
    y_pre = np.array(y_pre)
    m = y_test-y_pre
    m=np.square(m)
    m=m.sum(axis=1)
    m=np.sqrt(m)
    return m.sum()/len(y_test)

def get_model(feat_dim,output_dim = 2 ):
    net = nn.Sequential(
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim*4/5)),
        nn.BatchNorm1d(int(feat_dim*4/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*4/5),int(feat_dim*3/5)),
        nn.BatchNorm1d(int(feat_dim*3/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*3/5),int(feat_dim*2/5)),
        nn.BatchNorm1d(int(feat_dim*2/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*2/5),int(feat_dim/5)),
        nn.BatchNorm1d(int(feat_dim/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim/5), output_dim)
        )
    return net

def get_data(x,y,batch_size,shuffle):
    dataset = TensorDataset(x,y)
    return DataLoader(dataset, batch_size, shuffle=shuffle,num_workers=4)

def train_model(model,x_train,y_train,x_valid,y_valid,epochs,batch_size,lr,weight_decay,use_gpu):
    
    if use_gpu:
        model = model.cuda()
    train_data = get_data(x_train,y_train,batch_size,True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    #criterion = nn.CrossEntropyLoss()
    best_error = 0
    for e in  range(epochs):
        model.train()
        for data in train_data:
            x,y = data
            if use_gpu:
                x = x.cuda()
                y = y.cuda()
                
            out = model(x)
            loss = criterion(out, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if use_gpu:
                x_valid = x_valid.cuda()
        with torch.no_grad():
            output = model(x_valid)
        output = Tensor.cpu(output)
        out = output.numpy()
        print('epoch:{}'.format(e))
        error_i = error(out,y_valid)
        print('error:{}'.format(error_i)) 
        if e == 0:
            best_error = error_i
        elif error_i < best_error:
            best_model = copy.deepcopy(model)
    torch.save(best_model,"model_NN_master.t7")

def predict(x_test,use_gpu):
    net = torch.load('./model_NN_master.t7')
    if use_gpu:
        x_test = x_test.cuda()
    with torch.no_grad():
        a = Tensor.cpu(net(x_test)).numpy()
    #a = net(x_test).detach().numpy()
    #a = np.argmax(a,axis=1)
    #np.savetxt('NN_ans.txt',np.array(a),fmt='%f')
    #print(a)
    a = pd.DataFrame(a)
    a.to_csv('NN_base.csv',header=None)
    '''
    with open('./NN_master_ans.txt','w') as f:
        for i in a:
            f.write('{}'.format(i))
            f.write('\n')'''
    

In [ ]:
# 可以调整的超参
batch_size = 128
epochs = 40000
lr = 0.001
wd = 0
use_gpu = False
feat_dim = train_feat.shape[1]
net = get_model(feat_dim)
train_model(net, train_features, train_label, valid_features, valid_labels, epochs, 
            batch_size, lr, wd, use_gpu)
predict(x_test,use_gpu)

epoch:0
error:37.4615161101229
epoch:1
error:37.30321191785252
epoch:2
error:37.142969760672706
epoch:3
error:36.990946899256144
epoch:4
error:36.791230392949544
epoch:5
error:36.57967132923674
epoch:6
error:36.35823799320828
epoch:7
error:36.10925371927555
epoch:8
error:35.84254729948253
epoch:9
error:35.578958804980594
epoch:10
error:35.217188510672706
epoch:11
error:34.85699486578267
epoch:12
error:34.454173572930145
epoch:13
error:34.06142868693402
epoch:14
error:33.648247998868044
epoch:15
error:33.19947899822122
epoch:16
error:32.754972509702455
epoch:17
error:32.291862063389395
epoch:18
error:31.845831985769728
epoch:19
error:31.33686984556921
epoch:20
error:30.885364448576972
epoch:21
error:30.368279026520053
epoch:22
error:29.865095407503233
epoch:23
error:29.35192634217335
epoch:24
error:28.832743774256144
epoch:25
error:28.293271951811125
epoch:26
error:27.78018879366106
epoch:27
error:27.21260510996119
epoch:28
error:26.701116288001295
epoch:29
error:26.16752152732859
epoch

epoch:241
error:1.7793794659100097
epoch:242
error:1.8926665588615783
epoch:243
error:1.8179983753436286
epoch:244
error:1.778584666579075
epoch:245
error:1.7238842173350581
epoch:246
error:1.8915658731201488
epoch:247
error:1.8344653920399419
epoch:248
error:1.8421319761683377
epoch:249
error:1.8554496419691946
epoch:250
error:1.912687353452458
epoch:251
error:1.8139499994946637
epoch:252
error:1.7785132878193726
epoch:253
error:1.9113955874029753
epoch:254
error:1.9452570709189037
epoch:255
error:1.7848356772517788
epoch:256
error:1.794482390291478
epoch:257
error:1.9176683907058538
epoch:258
error:1.9962908311772316
epoch:259
error:1.972554393141575
epoch:260
error:1.9480049952498384
epoch:261
error:1.881310703529269
epoch:262
error:2.0247712721337323
epoch:263
error:1.947953514108991
epoch:264
error:1.8446968739893272
epoch:265
error:1.9692367020739003
epoch:266
error:2.0157462807244504
epoch:267
error:1.8799587708703913
epoch:268
error:1.7961814258570505
epoch:269
error:1.78621303

epoch:477
error:1.5920520698576972
epoch:478
error:1.6958802138078104
epoch:479
error:1.8246937661707632
epoch:480
error:1.688375021476795
epoch:481
error:1.6427297259055627
epoch:482
error:1.6400073842274419
epoch:483
error:1.6150047627951165
epoch:484
error:1.706424846377749
epoch:485
error:1.5659536581298512
epoch:486
error:1.7325696858829236
epoch:487
error:1.6434499881245956
epoch:488
error:1.671501051099612
epoch:489
error:1.5734929606646184
epoch:490
error:1.6153528132074708
epoch:491
error:1.6268924846377748
epoch:492
error:1.6030320812782988
epoch:493
error:1.5466438086190168
epoch:494
error:1.6403134285555465
epoch:495
error:1.7219797309589262
epoch:496
error:1.583898467820181
epoch:497
error:1.6070652970367076
epoch:498
error:1.6030140786707632
epoch:499
error:1.6116318005639554
epoch:500
error:1.6805418342698901
epoch:501
error:2.0745557461796573
epoch:502
error:1.8172008914133246
epoch:503
error:1.7489234756528946
epoch:504
error:1.7007751859637774
epoch:505
error:1.786651

epoch:713
error:1.4563736810721215
epoch:714
error:1.4379200608424967
epoch:715
error:1.494477621179657
epoch:716
error:1.428487641999515
epoch:717
error:1.5280161631832148
epoch:718
error:1.5067138671875
epoch:719
error:1.5058498999434022
epoch:720
error:1.450378181092335
epoch:721
error:1.5090107788243856
epoch:722
error:1.5755158852482212
epoch:723
error:1.685888450790346
epoch:724
error:1.6650112690006469
epoch:725
error:1.5345759027833927
epoch:726
error:1.4984651987487871
epoch:727
error:1.5142590126738358
epoch:728
error:1.5453431991833764
epoch:729
error:1.4852940804899741
epoch:730
error:1.4417978856727038
epoch:731
error:1.5096364483950517
epoch:732
error:1.5195492526075356
epoch:733
error:1.7425966645274094
epoch:734
error:1.5767624868612549
epoch:735
error:1.4554460730312095
epoch:736
error:1.4969089207026196
epoch:737
error:1.551981108000485
epoch:738
error:1.5274865075194048
epoch:739
error:1.5046295126536222
epoch:740
error:1.503861085563551
epoch:741
error:1.46043469032

epoch:950
error:1.4894927939036222
epoch:951
error:1.393726186024418
epoch:952
error:1.4096117500808538
epoch:953
error:1.4431321315592658
epoch:954
error:1.4379484860122898
epoch:955
error:1.3850983574041882
epoch:956
error:1.5362547248948901
epoch:957
error:1.4228994115358182
epoch:958
error:1.445914639846782
epoch:959
error:1.4528514866995472
epoch:960
error:1.4367135703023932
epoch:961
error:1.4228916735729302
epoch:962
error:1.3955378168458927
epoch:963
error:1.4099798560296732
epoch:964
error:1.441573168913729
epoch:965
error:1.579679856787678
epoch:966
error:1.516800854523771
epoch:967
error:1.504130177170925
epoch:968
error:1.4687877423087807
epoch:969
error:1.4846504083117724
epoch:970
error:1.4675086096680952
epoch:971
error:1.4750014844255337
epoch:972
error:1.4231394463029592
epoch:973
error:1.4314340687661709
epoch:974
error:1.4013501323981241
epoch:975
error:1.4319266137916398
epoch:976
error:1.4527314693159767
epoch:977
error:1.4263034835361417
epoch:978
error:1.50933324

...

epoch:20413
error:1.0850024698314198
epoch:20414
error:1.0581193654996766
epoch:20415
error:1.0891163025297137
epoch:20416
error:1.073880569438268
epoch:20417
error:1.0827566445413568
epoch:20418
error:1.1113717892191544
epoch:20419
error:1.0750991406755337
epoch:20420
error:1.1118135637330206
epoch:20421
error:1.0507438235264392
epoch:20422
error:1.0379665517868693
epoch:20423
error:1.0543656639108991
epoch:20424
error:1.0648950574314764
epoch:20425
error:1.0540307206601713
epoch:20426
error:1.1388435314420278
epoch:20427
error:1.1045376677716687
epoch:20428
error:1.16123490623484
epoch:20429
error:1.1768199530795198
epoch:20430
error:1.1238587294328104
epoch:20431
error:1.1762507390544146
epoch:20432
error:1.082552062277652
epoch:20433
error:1.1175232328387774
epoch:20434
error:1.0987257470134622
epoch:20435
error:1.0698302196191785
epoch:20436
error:1.0815619188631953
epoch:20437
error:1.0808029668297219
epoch:20438
error:1.0917777669691946
epoch:20439
error:1.064636467345165
epoch:

epoch:20636
error:1.09395276621119
epoch:20637
error:1.1012682994926424
epoch:20638
error:1.1221034751980918
epoch:20639
error:1.161409957905482
epoch:20640
error:1.097082061570181
epoch:20641
error:1.1014460357626537
epoch:20642
error:1.1363804115206582
epoch:20643
error:1.152457766514392
epoch:20644
error:1.1643134310822283
epoch:20645
error:1.0843409529632924
epoch:20646
error:1.0980366735223965
epoch:20647
error:1.0985952281088291
epoch:20648
error:1.1092312949749352
epoch:20649
error:1.1116276157472913
epoch:20650
error:1.0886854243309347
epoch:20651
error:1.1156568046066462
epoch:20652
error:1.1120704956844276
epoch:20653
error:1.0918589366207148
epoch:20654
error:1.0812647179212485
epoch:20655
error:1.0942716018657017
epoch:20656
error:1.1061640612366592
epoch:20657
error:1.0923634833845408
epoch:20658
error:1.0675623648225259
epoch:20659
error:1.0774794328104786
epoch:20660
error:1.09724898048391
epoch:20661
error:1.083256216900469
epoch:20662
error:1.1016465121684993
epoch:206

epoch:20859
error:1.0791051946050292
epoch:20860
error:1.1004018055667852
epoch:20861
error:1.0598136635369502
epoch:20862
error:1.1015737121503073
epoch:20863
error:1.0880554909847995
epoch:20864
error:1.0492278934296166
epoch:20865
error:1.0484841804455045
epoch:20866
error:1.080333793610527
epoch:20867
error:1.0651391190976713
epoch:20868
error:1.1019962996745634
epoch:20869
error:1.0753678374888826
epoch:20870
error:1.0641248142888906
epoch:20871
error:1.063533175963171
epoch:20872
error:1.0693578880477845
epoch:20873
error:1.053104297001334
epoch:20874
error:1.0681544769010753
epoch:20875
error:1.0737806075911627
epoch:20876
error:1.1222887125545764
epoch:20877
error:1.124320322606727
epoch:20878
error:1.1295990345548999
epoch:20879
error:1.1308282652308377
epoch:20880
error:1.0860535694433215
epoch:20881
error:1.0730968243400307
epoch:20882
error:1.325541625818645
epoch:20883
error:1.1570569589869018
epoch:20884
error:1.1513081263138745
epoch:20885
error:1.1417038931112549
epoch:

epoch:21082
error:1.1811640296531372
epoch:21083
error:1.1096036646992238
epoch:21084
error:1.1050324236436773
epoch:21085
error:1.1017264974379446
epoch:21086
error:1.0766568399801908
epoch:21087
error:1.084598674502749
epoch:21088
error:1.1309384917225906
epoch:21089
error:1.0557512330206986
epoch:21090
error:1.0635938163253962
epoch:21091
error:1.0927878080025064
epoch:21092
error:1.0692258689258571
epoch:21093
error:1.0810249989893272
epoch:21094
error:1.0938946525307245
epoch:21095
error:1.0803255818948092
epoch:21096
error:1.0965352718456904
epoch:21097
error:1.1257556357636642
epoch:21098
error:1.1266955613781533
epoch:21099
error:1.1017111794297785
epoch:21100
error:1.09033171541478
epoch:21101
error:1.1269400178383733
epoch:21102
error:1.1480657619461514
epoch:21103
error:1.1097537653874918
epoch:21104
error:1.0994663016453752
epoch:21105
error:1.0912946970003234
epoch:21106
error:1.1377547684801503
epoch:21107
error:1.2232523889776035
epoch:21108
error:1.1016491967678688
epoc

epoch:21305
error:1.0684465455206986
epoch:21306
error:1.0661141813803767
epoch:21307
error:1.0563495828448415
epoch:21308
error:1.060323263664295
epoch:21309
error:1.062882713327741
epoch:21310
error:1.0605934606949385
epoch:21311
error:1.0590922958946474
epoch:21312
error:1.0880417521527328
epoch:21313
error:1.064029668928889
epoch:21314
error:1.1017434735810154
epoch:21315
error:1.056689421541478
epoch:21316
error:1.096980836382196
epoch:21317
error:1.1075771080105918
epoch:21318
error:1.077266401954641
epoch:21319
error:1.1131975536667205
epoch:21320
error:1.0931527555991267
epoch:21321
error:1.0935154923492076
epoch:21322
error:1.307568864711352
epoch:21323
error:1.2002542947273205
epoch:21324
error:1.1553592657210139
epoch:21325
error:1.1233798442806031
epoch:21326
error:1.1335242346680952
epoch:21327
error:1.1182913440936286
epoch:21328
error:1.115470145991672
epoch:21329
error:1.0923754851228977
epoch:21330
error:1.105566264124151
epoch:21331
error:1.1088272627698497
epoch:2133

In [ ]:
y_valid = valid_labels#.cuda()
x_valid = valid_features.cuda()
net = torch.load('.//model_NN_master.t7')
with torch.no_grad():
    output_1 = net(x_valid)
output_1 = Tensor.cpu(output_1)  
out_1 = output_1.numpy()
error(out_1,y_valid)



In [25]:
predict(x_test,use_gpu)


In [11]:
from graphviz import Digraph
from torchviz import make_dot

In [19]:
make_dot(net(torch.randn(30,1)))


RuntimeError: size mismatch, m1: [30 x 1], m2: [30 x 30] at /opt/conda/conda-bld/pytorch_1549628766161/work/aten/src/TH/generic/THTensorMath.cpp:940